In [2]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## Acurácia:

#### Certos / Total

In [6]:
previu_certo = 80
previu_errado = 20

acuracia = previu_certo/(previu_certo+previu_errado)
acuracia

0.8

# O que é melhor?

## Um modelo de predição de cancer que tem 80% de acurácia, ou

## Um modelo de predição de cancer que tem 99,60% de acurácia?

In [10]:
# Resposta = ...

In [7]:
# Problema:

pessoas_com_cancer = 20
pessoas_sem_cancer = 5000

In [8]:
5000/5020

0.9960159362549801

In [104]:
pessoas_sem_cancer = np.zeros(1000)
pessoas_com_cancer = np.ones(150)

y_test = list(pessoas_com_cancer) + list(pessoas_sem_cancer)

In [105]:
y_pred = np.zeros(1150) #Nosso preditor burro

In [106]:
a = np.ones(100)
b = np.zeros(1150-100-150)
c = np.ones(150)
y_pred2 = list(a) + list(b) + list(c) # Nosso preditor com menor acurácia

In [111]:
print("Acuracidade do modelo:", accuracy_score(y_pred, y_test), '\n')
print(classification_report(y_pred, y_test))
print("Matriz de Confusão: \n\n",confusion_matrix(y_pred, y_test))

Acuracidade do modelo: 0.8695652173913043 

              precision    recall  f1-score   support

         0.0       1.00      0.87      0.93      1150
         1.0       0.00      0.00      0.00         0

   micro avg       0.87      0.87      0.87      1150
   macro avg       0.50      0.43      0.47      1150
weighted avg       1.00      0.87      0.93      1150

Matriz de Confusão: 

 [[1000  150]
 [   0    0]]


In [112]:
print("Acuracidade do modelo:", accuracy_score(y_pred2, y_test), '\n')
print(classification_report(y_pred2, y_test))
print("Matriz de Confusão: \n\n",confusion_matrix(y_pred2, y_test))

Acuracidade do modelo: 0.8260869565217391 

              precision    recall  f1-score   support

         0.0       0.85      0.94      0.89       900
         1.0       0.67      0.40      0.50       250

   micro avg       0.83      0.83      0.83      1150
   macro avg       0.76      0.67      0.70      1150
weighted avg       0.81      0.83      0.81      1150

Matriz de Confusão: 

 [[850  50]
 [150 100]]


## Matriz de Confusão:

![metricas](https://miro.medium.com/max/863/1*0exdQRxrXQgIBZdPFIxbTw.png)